<a href="https://colab.research.google.com/github/veruizr/ML_Doc/blob/main/ejercicio1_clase2702.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#cliente	productos comprados
  #1	        a,b,c
  #2	        a,b
  #3	        a,c,d
  #4	        a
  #5	        b,c
  #6	        b,c,d


import polars as pl

#Datos de clientes y productos

data = {"Cliente": [1, 2, 3, 4, 5, 6],
"Productos_comprados": ["a,b,c", "a,b", "a,c,d", "a", "b,c", "b,c,d"]}

df = pl.DataFrame(data)

#crear una tabla binaria a partir de los datos

#extraer la lista de los rpoductos individuales
productos = sorted(set(
producto for lista in df["Productos_comprados"].to_list() for producto in lista.split(",")
))

# productos en columnas binarias

df_bin = df.with_columns([
df["Productos_comprados"].str.contains(p).cast(pl.Int8).alias(p) for p in productos
]).drop("Productos_comprados")

#Cálculo de la probabilidad de compra de cada producto
# Número total de clientes
total_clientes = df_bin.height

#probabilidad claculada como Probabilidad de producto/No clientes
probabilidades = df_bin.drop("Cliente").sum() / total_clientes
#organizar una tabla de productos y sus probabilidades
probabilidades = probabilidades.transpose(include_header=True, header_name="Probabilidad")
probabilidades.columns = ["Producto", "Probabilidad"]

#Identificar productos con probabilidad mayor al 50%

prob_mayores_50 = probabilidades.filter(pl.col("Probabilidad") > 0.5)

print("Probabilidades de compra mayores al 50%:")
print(prob_mayores_50)

Probabilidades de compra mayores al 50%:
shape: (3, 2)
┌──────────┬──────────────┐
│ Producto ┆ Probabilidad │
│ ---      ┆ ---          │
│ str      ┆ f64          │
╞══════════╪══════════════╡
│ a        ┆ 0.666667     │
│ b        ┆ 0.666667     │
│ c        ┆ 0.666667     │
└──────────┴──────────────┘


In [24]:
df_bin

Cliente,a,b,c,d
i64,i8,i8,i8,i8
1,1,1,1,0
2,1,1,0,0
3,1,0,1,1
4,1,0,0,0
5,0,1,1,0
6,0,1,1,1
